In [21]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [31]:
out='../output/'
inb='../input/flows/'
pref='fpm'
inp = inb + pref + '/'

In [32]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-04-01,10,BOT,0,ZIMA
1,2016-04-01,10,MOZN_EDM,0,ZIMA
2,2016-04-01,10,MOZN_HCB,0,MOZN_EDM
3,2016-04-01,10,MOZS,0,RSAN
4,2016-04-01,10,RSAN,0,BOT
...,...,...,...,...,...
2275,2021-05-31,20,MOZS,23,RSAN
2276,2021-05-31,20,RSAN,23,BOT
2277,2021-05-31,20,ZAMZ,23,ZAMC
2278,2021-05-31,20,ZIM,23,ZAMZ


In [24]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week      value  \
date       hour                                                             
2016-01-01 0      40.10   477.5  2016      1    1            4  19147.750   
           1      40.08   486.6  2016      1    1            4  19502.928   
           2      40.06   495.6  2016      1    1            4  19853.736   
           3      40.08   496.6  2016      1    1            4  19903.728   
           4      40.07   502.9  2016      1    1            4  20151.203   
...                 ...     ...   ...    ...  ...          ...        ...   
2021-05-31 19    288.09    53.8  2021      5   31            0  15499.242   
           20    118.04    60.0  2021      5   31            0   7082.400   
           21     79.60    73.0  2021      5   31            0   5810.800   
           22     41.15   126.0  2021      5   31            0   5184.900   
           23     41.11    96.0  2021      5   31            0   3946.560   

                      cat  
date       hour            
2016-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2021-05-31 19     evening  
           20    standard  
           21    standard  
           22         off  
           23         off  

[46840 rows x 8 columns]

In [25]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.00
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.00
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.80
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.60
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.60
...,...,...,...,...,...,...,...,...,...,...,...
339146,2021-05-31,23,61,RSAS,NAM,2021,5,31,0,41.11,2507.71
339147,2021-05-31,23,35,ZAMZ,ZAMC,2021,5,31,0,41.11,1438.85
339148,2021-05-31,23,35,ZIM,ZAMZ,2021,5,31,0,41.11,1438.85
339149,2021-05-31,23,35,ZIMA,ZIM,2021,5,31,0,41.11,1438.85


In [26]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.00,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.00,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.80,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.60,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.60,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
339146,2021-05-31,23,-61,-2507.71,41.11,RSAS,2021,5,31,0
339147,2021-05-31,23,-35,-1438.85,41.11,ZAMZ,2021,5,31,0
339148,2021-05-31,23,-35,-1438.85,41.11,ZIM,2021,5,31,0
339149,2021-05-31,23,-35,-1438.85,41.11,ZIMA,2021,5,31,0


In [27]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [28]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week   index  flow    value  \
date       hour                                                               
2016-01-01 0     ZAMC  2016      1    1            4       0   160  6416.00   
           0     ZAMZ  2016      1    1            4       1    10   401.00   
           0      ZIM  2016      1    1            4       1  -170 -6817.00   
           1     ZAMC  2016      1    1            4       2   160  6412.80   
           1     ZAMZ  2016      1    1            4       5    10   400.80   
...               ...   ...    ...  ...          ...     ...   ...      ...   
2021-05-31 22     NAM  2021      5   31            0  339137    91  3744.65   
           22    ZAMC  2021      5   31            0  339138    35  1440.25   
           23     MOZ  2021      5   31            0  678287   -96 -3946.56   
           23     NAM  2021      5   31            0  339146    61  2507.71   
           23    ZAMC  2021      5   31            0  339147    35  1438.85   

                 price  cat  
date       hour              
2016-01-01 0     40.10  off  
           0     80.20  off  
           0     40.10  off  
           1     40.08  off  
           1     80.16  off  
...                ...  ...  
2021-05-31 22    41.15  off  
           22    41.15  off  
           23    82.22  off  
           23    41.11  off  
           23    41.11  off  

[199034 rows x 10 columns]

In [29]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2016 1     BOT  evening  supply        -25   -1501.40           2   
                morning  supply       -285  -14572.40           4   
                off      supply      -1270  -59474.14          45   
                standard supply      -1165  -57809.40          18   
           HCB  evening  supply        -57   -6204.59           2   
...                                    ...        ...         ...   
2021 5     ZIM  morning  consume         2     339.72           2   
                         supply       -417  -60998.66          21   
                off      consume      4069  139393.79          92   
                standard consume        15     571.33           3   
                         supply       -536  -62423.39          24   

                                  price_weight_mean  price_mean      power  
year month node cat      side                                               
2016 1     BOT  evening  supply           60.056000   60.050000 -12.500000  
                morning  supply           51.131228   55.290000 -71.250000  
                off      supply           46.830031   48.463556 -28.222222  
                standard supply           49.621803   56.768333 -64.722222  
           HCB  evening  supply          108.852456   99.140000 -28.500000  
...                                             ...         ...        ...  
2021 5     ZIM  morning  consume         169.860000  849.300000   1.000000  
                         supply          146.279760  652.396667 -19.857143  
                off      consume          34.257506  142.046630  44.228261  
                standard consume          38.088667  227.003333   5.000000  
                         supply          116.461549  441.631667 -22.333333  

[2397 rows x 6 columns]

In [30]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')